<a href="https://colab.research.google.com/github/RohanRaj094/NLP_CV/blob/main/CV_Assignment_4(Final_Project).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<style>
blue {
  color: skyblue;
}
green {
  color: lightgreen;
}
</style>
### **Step - 1: Install All Libraries**
This cell installs all the libraries we need.
1. <blue>**fiftyone**</blue>: The library we use to download the <green>COCO dataset</green>.
2. <blue>**segment-anything**</blue>: The official library from Meta to run the <green>SAM model</green>.
3. <blue>**pycocotools**</blue>: A helper library that <green>fiftyone</green> needs to load COCO.

In [1]:
!pip install -q fiftyone supervision
!pip install -q git+https://github.com/facebookresearch/segment-anything.git
!pip install -q pycocotools

print("All libraries installed.")

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 133.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.2/207.2 kB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.6/112.6 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.7/61.7 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.4/112.4 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.8/74.8 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 112.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 76.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 308.7/308.7 kB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 934.3/934.3 kB 53.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 3.9 MB/s eta 

<style>
blue {
  color: skyblue;
}
green {
  color: lightgreen;
}
</style>
### **Step - 2: Import Libraries & Set Device**

In [2]:
import torch
import torchvision
import fiftyone as fo
import fiftyone.zoo as foz
import supervision as sv
import numpy as np
import time
import requests
import os
from segment_anything import SamAutomaticMaskGenerator, sam_model_registry
from PIL import Image
import matplotlib.pyplot as plt

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {DEVICE}")

/usr/local/lib/python3.12/dist-packages/glob2/fnmatch.py:141: SyntaxWarning: invalid escape sequence '\Z'
  return '(?ms)' + res + '\Z'


Using device: cuda


<style>
blue {
  color: skyblue;
}
green {
  color: lightgreen;
}
</style>
### **Step - 3: Download Dataset**
1. <blue>**foz.load_zoo_dataset**</blue>: This function downloads the <green>validation split</green> of COCO.
2. <blue>**max_samples=500**</blue>: Dataset size to <green>500 images</green>.

In [3]:
dataset_name = "coco-2017-validation-500"

if fo.dataset_exists(dataset_name):
    fo.delete_dataset(dataset_name)

dataset = foz.load_zoo_dataset(
    "coco-2017",
    split="validation",
    label_types=["segmentations"],
    max_samples=500,
    dataset_name=dataset_name
)
dataset.persistent = True
print(f"Loaded {len(dataset)} images.")

You are running the oldest supported major version of MongoDB. Please refer to https://deprecation.voxel51.com for deprecation notices. You can suppress this exception by setting your `database_validation` config parameter to `False`. See https://docs.voxel51.com/user_guide/config.html#configuring-a-mongodb-connection for more information


INFO:fiftyone.zoo.datasets:Downloading split 'validation' to '/root/fiftyone/coco-2017/validation' if necessary


INFO:fiftyone.utils.coco:Downloading annotations to '/root/fiftyone/coco-2017/tmp-download/annotations_trainval2017.zip'


 100% |██████|    1.9Gb/1.9Gb [5.0s elapsed, 0s remaining, 436.9Mb/s]       


INFO:eta.core.utils: 100% |██████|    1.9Gb/1.9Gb [5.0s elapsed, 0s remaining, 436.9Mb/s]       


...

 100% |█████████████████| 500/500 [9.8s elapsed, 0s remaining, 66.0 samples/s]      


INFO:eta.core.utils: 100% |█████████████████| 500/500 [9.8s elapsed, 0s remaining, 66.0 samples/s]      


Dataset 'coco-2017-validation-500' created


INFO:fiftyone.zoo.datasets:Dataset 'coco-2017-validation-500' created


Loaded 500 images.


<style>
blue {
  color: skyblue;
}
green {
  color: lightgreen;
}
</style>
### **Step - 4: Load Models**
1. <blue>**Mask R-CNN**</blue>: We load the <green>pre-trained</green> model from `torchvision`.
2. <blue>**SAM**</blue>: We download the official <green>SAM checkpoint file</green> and use the `sam_model_registry`.

In [4]:
mask_rcnn_model = torchvision.models.detection.maskrcnn_resnet50_fpn(weights='DEFAULT').to(DEVICE)
mask_rcnn_model.eval()

checkpoint_url = "https://dl.fbaipublicfiles.com/segment_anything/sam_vit_b_01ec64.pth"
checkpoint_path = "sam_vit_b_01ec64.pth"

if not os.path.exists(checkpoint_path):
    print("Downloading SAM checkpoint...")
    r = requests.get(checkpoint_url)
    with open(checkpoint_path, 'wb') as f:
        f.write(r.content)
    print("Download complete.")

model_type = "vit_b"
sam_model = sam_model_registry[model_type](checkpoint=checkpoint_path)
sam_model.to(device=DEVICE)

sam_mask_generator = SamAutomaticMaskGenerator(sam_model)

print("Both models are loaded.")

Downloading: "https://download.pytorch.org/models/maskrcnn_resnet50_fpn_coco-bf2d0c1e.pth" to /root/.cache/torch/hub/checkpoints/maskrcnn_resnet50_fpn_coco-bf2d0c1e.pth


100%|██████████| 170M/170M [00:01<00:00, 172MB/s]


Download complete.
Both models are loaded.


<style>
blue {
  color: skyblue;
}
green {
  color: lightgreen;
}
</style>
### **Step - 5: Helper Functions**
This block defines small functions we'll need in the main loop.
1. <blue>**preprocess**</blue>: Prepares images for the <green>Mask R-CNN</green> model.
2. <blue>**get_ground_truth_masks**</blue>: Gets the <green>correct "answer" masks</green> from the dataset and stacks them into a single tensor.
3. <blue>**calculate_dice_score**</blue>: The function to measure <green>mask overlap</green>.

In [5]:
preprocess = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
])

COCO_CLASSES = [
    '__background__', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus',
    'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'N/A', 'stop sign',
    'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow',
    'elephant', 'bear', 'zebra', 'giraffe', 'N/A', 'backpack', 'umbrella', 'N/A', 'N/A',
    'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
    'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket',
    'bottle', 'N/A', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl',
    'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
    'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'N/A', 'dining table',
    'N/A', 'N/A', 'toilet', 'N/A', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone',
    'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'N/A', 'book',
    'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush'
]

def calculate_dice_score(mask1, mask2):
    mask1 = mask1.bool()
    mask2 = mask2.bool()

    intersection = (mask1 & mask2).sum().float()
    union = mask1.sum().float() + mask2.sum().float()

    if union == 0:
        return 1.0

    dice = (2. * intersection) / (union + 1e-8)
    return dice.item()

def get_ground_truth_masks(sample):
    img_height = sample.metadata.height
    img_width = sample.metadata.width
    gt_masks = []

    if sample.ground_truth is None:
        return torch.empty(0, img_height, img_width)

    for detection in sample.ground_truth.detections:
        if detection.mask is not None:
            full_mask = np.zeros((img_height, img_width), dtype=np.uint8)

            cropped_mask = detection.mask
            h_crop, w_crop = cropped_mask.shape

            x, y, w, h = detection.bounding_box
            xmin = int(x * img_width)
            ymin = int(y * img_height)

            ymax = ymin + h_crop
            xmax = xmin + w_crop

            # Check for rounding errors at the image boundary
            if ymax > img_height:
                ymax = img_height
            if xmax > img_width:
                xmax = img_width

            # Slice the cropped mask if it goes over the boundary
            paste_h = ymax - ymin
            paste_w = xmax - xmin

            full_mask[ymin:ymax, xmin:xmax] = cropped_mask[0:paste_h, 0:paste_w]

            gt_masks.append(torch.from_numpy(full_mask))

    if not gt_masks:
        return torch.empty(0, img_height, img_width)

    return torch.stack(gt_masks)

print("Helper functions defined.")

Helper functions defined.


<style>
blue {
  color: skyblue;
}
green {
  color: lightgreen;
}
</style>
### **Step - 6: Run Evaluation Loop**
 It loops over all <green>**500 images**</green> to run the comparison.
1. <blue>**For loop**</blue>: Iterates through each `sample` in our `dataset`.
2. <blue>**Mask R-CNN / SAM**</blue>: Runs each model on the image, <green>times the speed</green>, and calculates the `dice_score`.
3. <blue>**Results**</blue>: At the end, it prints the <green>final average Dice Score</green> and <green>Inference Speed (FPS)</green>.

In [ ]:
results = []
mask_rcnn_total_time = 0
sam_total_time = 0

num_images = len(dataset)
print(f"Found {num_images} images to evaluate.")

print("Starting evaluation loop...")

with fo.ProgressBar() as pb:
    for sample in dataset.iter_samples(autosave=True, progress=pb):
        image_path = sample.filepath
        image_pil = Image.open(image_path).convert("RGB")
        image_np = np.array(image_pil)

        gt_masks = get_ground_truth_masks(sample).to(DEVICE)

        image_tensor = preprocess(image_pil).to(DEVICE)

        start_time = time.time()
        with torch.no_grad():
            prediction = mask_rcnn_model([image_tensor])
        mask_rcnn_total_time += (time.time() - start_time)

        pred_masks = prediction[0]['masks'][:, 0, :, :] > 0.5

        avg_dice_rcnn = 0
        if gt_masks.shape[0] > 0 and pred_masks.shape[0] > 0:
            dice_scores = []
            for gt_mask in gt_masks:
                best_dice = 0
                for pred_mask in pred_masks:
                    dice = calculate_dice_score(gt_mask, pred_mask)
                    if dice > best_dice:
                        best_dice = dice
                dice_scores.append(best_dice)
            avg_dice_rcnn = np.mean(dice_scores) if dice_scores else 0


        start_time = time.time()
        outputs = sam_mask_generator.generate(image_np)
        sam_total_time += (time.time() - start_time)

        sam_masks = [torch.from_numpy(mask['segmentation']).to(DEVICE) for mask in outputs]
        if sam_masks:
            sam_masks_tensor = torch.stack(sam_masks)
        else:
            sam_masks_tensor = torch.empty(0, image_np.shape[0], image_np.shape[1]).to(DEVICE)

        avg_dice_sam = 0
        if gt_masks.shape[0] > 0 and sam_masks_tensor.shape[0] > 0:
            dice_scores = []
            for gt_mask in gt_masks:
                best_dice = 0
                for pred_mask in sam_masks_tensor:
                    dice = calculate_dice_score(gt_mask, pred_mask)
                    if dice > best_dice:
                        best_dice = dice
                dice_scores.append(best_dice)
            avg_dice_sam = np.mean(dice_scores) if dice_scores else 0

        results.append({
            "image_id": sample.id,
            "rcnn_dice": avg_dice_rcnn,
            "sam_dice": avg_dice_sam
        })

print("Evaluation complete.")

avg_rcnn_dice = np.mean([r['rcnn_dice'] for r in results]) if results else 0
avg_sam_dice = np.mean([r['sam_dice'] for r in results]) if results else 0
avg_rcnn_fps = num_images / mask_rcnn_total_time if mask_rcnn_total_time > 0 else 0
avg_sam_fps = num_images / sam_total_time if sam_total_time > 0 else 0

print("\n--- Overall Results ---")
print(f"Mask R-CNN Avg. Dice Score: {avg_rcnn_dice:.4f}")
print(f"SAM Avg. Dice Score:          {avg_sam_dice:.4f}")
print(f"Mask R-CNN Avg. FPS: {avg_rcnn_fps:.2f} (Inference Speed)")
print(f"SAM Avg. FPS:          {avg_sam_fps:.2f} (Inference Speed)")

Found 500 images to evaluate.
Starting evaluation loop...
  16% |██\--------------|  79/500 [7.2m elapsed, 38.7m remaining, 0.2 samples/s]  

<style>
blue {
  color: skyblue;
}
green {
  color: lightgreen;
}
</style>
### **Step - 7: Visualize the Results**

In [ ]:
import random

sample_id = random.choice([r['image_id'] for r in results])
sample = dataset[sample_id]
image_path = sample.filepath
image_pil = Image.open(image_path).convert("RGB")
image_np = np.array(image_pil)

print(f"Visualizing results for image: {sample.filepath}")

gt_masks = get_ground_truth_masks(sample).cpu().numpy()

image_tensor = preprocess(image_pil).to(DEVICE)
with torch.no_grad():
    prediction = mask_rcnn_model([image_tensor])

rcnn_masks = prediction[0]['masks'][:, 0, :, :].cpu().numpy() > 0.5
rcnn_labels = [COCO_CLASSES[label] for label in prediction[0]['labels'].cpu().numpy()]
rcnn_scores = prediction[0]['scores'].cpu().numpy()

score_threshold = 0.7
rcnn_masks = rcnn_masks[rcnn_scores > score_threshold]
rcnn_labels = [label for i, label in enumerate(rcnn_labels) if rcnn_scores[i] > score_threshold]

outputs = sam_mask_generator.generate(image_np)
sam_masks = [mask['segmentation'] for mask in outputs]

fig, axes = plt.subplots(1, 3, figsize=(20, 7))

axes[0].imshow(image_np)
if gt_masks.shape[0] > 0:
    for mask in gt_masks:
        sv.plot_mask(ax=axes[0], mask=mask, random_color=True)
axes[0].set_title("Ground Truth")
axes[0].axis('off')

axes[1].imshow(image_np)
if rcnn_masks.shape[0] > 0:
    for i, mask in enumerate(rcnn_masks):
        sv.plot_mask(ax=axes[1], mask=mask, random_color=True)
        y, x = np.where(mask)
        if len(y) > 0:
             axes[1].text(np.mean(x), np.mean(y), rcnn_labels[i], color='white', fontsize=9, bbox=dict(facecolor='black', alpha=0.5))
axes[1].set_title(f"Mask R-CNN (Score > 0.7)")
axes[1].axis('off')

axes[2].imshow(image_np)
if sam_masks:
    for mask in sam_masks:
        sv.plot_mask(ax=axes[2], mask=mask, random_color=True)
axes[2].set_title(f"SAM (Zero-Shot)")
axes[2].axis('off')

plt.tight_layout()
plt.show()